In [29]:
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML

%pylab inline
# pip install git+git://github.com/mgelbart/plot-classifier.git
#!pip install git+https://github.com/mgelbart/plot-classifier.git
from plot_classifier import plot_classifier
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
    TfidfVectorizer,
)

# train test split and cross validation
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)

from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB, CategoricalNB, ComplementNB

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import classification_report

df=pd.read_csv("/Users/komal/Library/CloudStorage/OneDrive-UniversityofSouthampton/Postgraduate/E-books/BAMS/Advanced Analytics/purchase_data_regular.csv")
df.head()

Populating the interactive namespace from numpy and matplotlib


/Users/komal/opt/anaconda3/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['clf', 'uniform', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


,label,platform,prior_purchase,premium,redeem_card
0,0,1,3,2,2
1,0,2,3,2,2
2,0,3,1,2,2
3,0,1,3,2,2
4,0,2,1,2,2


In [3]:
#Question 1: Use Naïve Bayes reasoning, what is the probability that a customer purchases, given it is an Android user?
y=df['label']
X=np.array(df['platform']).reshape(-1, 1)
NB_model=[GaussianNB(),CategoricalNB(),BernoulliNB(),ComplementNB(),MultinomialNB()]
for model in NB_model:
    clf=model.fit(X,y)
    x=np.array([1]).reshape(-1,1)
    prob=clf.predict_proba(x)
    print(model,prob[0])

GaussianNB() [0.56835483 0.43164517]
CategoricalNB() [0.53785562 0.46214438]
BernoulliNB() [0.49398807 0.50601193]
ComplementNB() [0.5 0.5]
MultinomialNB() [0.494 0.506]


In [4]:
#Question 2: Use Naïve Bayes reasoning, what is the probability that a customer purchases, given it is an Android user and had a purchase in “between 6 and 12 months”?
y=df['label']
X=df[['platform','prior_purchase']]
NB_model=[GaussianNB(),CategoricalNB(),BernoulliNB(),ComplementNB(),MultinomialNB()]
for model in NB_model:
    clf=model.fit(X,y)
    x=X.loc[(X['platform']==1) & (X['prior_purchase']==3)]
    prob=clf.predict_proba(x)
    print(model,prob[0])

GaussianNB() [0.41693115 0.58306885]
CategoricalNB() [0.43865814 0.56134186]
BernoulliNB() [0.49397614 0.50602386]
ComplementNB() [0.48760307 0.51239693]
MultinomialNB() [0.48160854 0.51839146]


In [5]:
# y=df['label']
# X=df.drop(columns=['label'])
# X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=32841137)
# from sklearn.linear_model import LogisticRegression
# gnb=LogisticRegression().fit(X_train,y_train)

# x=X[X['platform']==1]
# #print(x)
# prob=gnb.predict_proba(x)
# y_pred=gnb.predict(X_test)
# print('LR1 prob:',gnb.predict_proba(x)[0])
# print('LR1 classification report:', classification_report(y_test, y_pred))
# print('LR1 accuracy:', accuracy_score(y_test, y_pred))


# x=X[X['platform']==1]
# #print(x)
# prob=X.loc[(X['platform']==1) & (X['prior_purchase']==3)]
# y_pred=gnb.predict(X_test)
# print('LR2 prob:',gnb.predict_proba(x)[0])
# print('LR2 classification report:', classification_report(y_test, y_pred))
# print('LR2 accuracy:', accuracy_score(y_test, y_pred))


In [6]:
#Question 3:
#SVM
y=df['label']
X=df.drop(columns=['label'])
distributions = dict(C=uniform(loc=0, scale=4))
model=SVC(kernel='linear')
acc_values_stratified=np.zeros(5)
acc_values_tune=np.zeros(10)
for itr in range(0,10):
    print(itr)
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=itr)
    for i,(train,test) in enumerate(skf.split(X,y)):
        clf=RandomizedSearchCV(model,distributions,cv=5,random_state=0)
        search=clf.fit(X.loc[train,:],y[train])
        acc_values_stratified[i]=search.score(X.loc[test,:],y[test])
    acc_values_tune[itr]=np.mean(acc_values_stratified)
np.mean(acc_values_tune)

0
1
2
3
4
5
6
7
8
9


0.6411

In [15]:
#GaussianNB
y=df['label']
X=df.drop(columns=['label'])
nb_classifier = GaussianNB()
for itr in range(0,10):
    print(itr)
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=32841337)
    params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
    gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB, 
                 cv=skf,   # use any cross validation technique 
                 verbose=1, 
                 scoring='accuracy') 
    for i,(train,test) in enumerate(skf.split(X,y)):
        search=gs_NB.fit(X.loc[train,:],y[train])
        #gs_NB.best_params_
        acc_values_stratified[i]=search.score(X.loc[test,:],y[test])
        acc_values_tune[itr]=np.mean(acc_values_stratified)
np.mean(acc_values_tune)

0
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
1
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
2
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
3
Fitting 5 folds for each of 100 candidates, tot

0.659

In [8]:
from sklearn.linear_model import LogisticRegression
y=df['label']
X=df.drop(columns=['label'])
skf = StratifiedKFold(n_splits=5)
acc_values_Stratified = np.zeros(5)

for i, (train, test) in enumerate(skf.split(X, y)):
    print(i)
    clf = LogisticRegression().fit(X.loc[train,:], y[train])
    acc_values_Stratified[i] = clf.score(X.loc[test,:], y[test])
np.mean(acc_values_Stratified)

0
1
2
3
4


0.643

In [9]:
y=df['label']
X=df.drop(columns=['label'])
distributions = dict(C=uniform(loc=0, scale=4))
model=LogisticRegression()
acc_values_stratified=np.zeros(5)
acc_values_tune=np.zeros(10)
for itr in range(0,10):
    print(itr)
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=itr)
    for i,(train,test) in enumerate(skf.split(X,y)):
        clf=RandomizedSearchCV(model,distributions,cv=5,random_state=0)
        search=clf.fit(X.loc[train,:],y[train])
        acc_values_stratified[i]=search.score(X.loc[test,:],y[test])
    acc_values_tune[itr]=np.mean(acc_values_stratified)
    print(acc_values_tune[itr])
print(np.mean(acc_values_tune))

0
0.645
1
0.644
2
0.641
3
0.628
4
0.644
5
0.646
6
0.64
7
0.639
8
0.641
9
0.6330000000000001
0.6401000000000001


In [35]:
from sklearn.ensemble import RandomForestClassifier
y=df['label']
X=df.drop(columns=['label'])  
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=32841337)
forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(X_train, y_train)
y_pred=forest.predict(X_test)
print('Train score:{:.3f}'.format(forest.score(X_train,y_train)))
print('Test score:{:.3f}'.format(forest.score(X_test,y_test)))
print(classification_report(y_test,y_pred,target_names=['not 1','1']))

Train score:0.721
Test score:0.708
              precision    recall  f1-score   support

       not 1       0.70      0.72      0.71       123
           1       0.72      0.69      0.71       127

    accuracy                           0.71       250
   macro avg       0.71      0.71      0.71       250
weighted avg       0.71      0.71      0.71       250



In [43]:
from sklearn.neighbors import KNeighborsClassifier
y=df['label']
X=df.drop(columns=['label'])  
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=32841337)
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)
print('Train score:{:.3f}'.format(knn.score(X_train,y_train)))
print('Test score:{:.3f}'.format(knn.score(X_test,y_test)))
print(classification_report(y_test,y_pred,target_names=['not 1','1']))

Train score:0.691
Test score:0.760
              precision    recall  f1-score   support

       not 1       0.78      0.72      0.75       123
           1       0.74      0.80      0.77       127

    accuracy                           0.76       250
   macro avg       0.76      0.76      0.76       250
weighted avg       0.76      0.76      0.76       250



In [50]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
y=df['label']
X=df.drop(columns=['label'])  
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=32841337)
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred=dtree.predict(X_test)
print('Train score:{:.3f}'.format(dtree.score(X_train,y_train)))
print('Test score:{:.3f}'.format(dtree.score(X_test,y_test)))
print('Test score:{:.3f}'.format(model.score(X_test,y_test)))
print(classification_report(y_test,y_pred,target_names=['not 1','1']))

Train score:0.721
Test score:0.704
Test score:0.644
              precision    recall  f1-score   support

       not 1       0.69      0.72      0.71       123
           1       0.72      0.69      0.70       127

    accuracy                           0.70       250
   macro avg       0.70      0.70      0.70       250
weighted avg       0.70      0.70      0.70       250



In [49]:

y=df['label']
X=df.drop(columns=['label'])  
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=32841337)
model=SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('Train score:{:.3f}'.format(model.score(X_train,y_train)))
print('Test score:{:.3f}'.format(model.score(X_test,y_test)))
print(classification_report(y_test,y_pred,target_names=['not 1','1']))

Train score:0.644
Test score:0.644
              precision    recall  f1-score   support

       not 1       0.66      0.58      0.61       123
           1       0.63      0.71      0.67       127

    accuracy                           0.64       250
   macro avg       0.65      0.64      0.64       250
weighted avg       0.65      0.64      0.64       250



In [48]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
y=df['label']
X=df.drop(columns=['label'])  
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=32841337)
model=LogisticRegression()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('Train score:{:.3f}'.format(model.score(X_train,y_train)))
print('Test score:{:.3f}'.format(model.score(X_test,y_test)))
print(classification_report(y_test,y_pred,target_names=['not 1','1']))
print('F1:{:.2f}'.format(f1_score(y_test,y_pred)))

Train score:0.644
Test score:0.652
              precision    recall  f1-score   support

       not 1       0.63      0.72      0.67       123
           1       0.68      0.59      0.63       127

    accuracy                           0.65       250
   macro avg       0.66      0.65      0.65       250
weighted avg       0.66      0.65      0.65       250

F1:0.63
